In [31]:
import os
import pandas as pd
import openai
import numpy as np
import tensorflow as tf
import regex as re
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Embedding
import json

pd.set_option('display.max_colwidth', None)

In [32]:
basepath = '../../datasets/preprocessed_datasets/retailrocket/'
product_data = pd.read_pickle(basepath +'item_data_extracted_10k.pkl')
product_data = product_data.reset_index(drop=True)
#product_data = product_data.sample(n=100, random_state=2).reset_index(drop=True)
#product_data.to_pickle(basepath +'item_data_extracted_10k.pkl')
display(product_data)

,itemid,categoryid,available,properties,property_values
0,309858,799,0,"[1066, 112, 159, 202, 227, 283, 364, 384, 400, 566, 595, 6, 678, 689, 698, 764, 776, 790, 839, 853, 888, 917, 934, 964, 992, 997]","[n1260.000 424566, 679677, 519769, 125782 n1440.000 822075, 1308244, 1308244 590597 125782 n1440.000 822075, 909382, 726612, n586.800 424566, n619.200 424566, 873905, 1308244, 590597, 150169 274770 79212, 1308244, 1285872, 1081162, n161400.000, 590597, 826838, 125782 n1440.000 822075, 1301157 n1440.000 822075, 638115, n2640.000 1128577, n21.600 404373, n360.000 863188]"
1,89393,491,0,"[0, 1036, 112, 159, 202, 208, 225, 227, 283, 364, 451, 6, 653, 663, 678, 681, 689, 696, 698, 761, 764, 776, 790, 794, 839, 888, 893, 917, 936, 969]","[[n24.000 266119 768453 119932 754228 98606 632686 234274 87299 386093 474154 1284577 155680, n24.000 266119 768453 119932 754228 98606 632686 234274 87299 386093 474154 1284577 155680, n24.000 266119 768453 119932 754228 98606 632686 234274 87299 386093 474154 1284577 155680, n24.000 266119 768453 119932 754228 98606 632686 234274 87299 386093 474154 1284577 155680, n24.000 266119 768453 119932 754228 98606 632686 234274 87299 386093 474154 1284577 155680, n24.000 266119 768453 119932 754228 98606 632686 234274 87299 386093 474154 1284577 155680, n24.000 266119 768453 119932 754228 98606 632686 234274 87299 386093 474154 1284577 155680, n24.000 266119 768453 119932 754228 98606 632686 234274 87299 386093 474154 1284577 155680, n24.000 266119 768453 119932 754228 98606 632686 234274 87299 386093 474154 1284577 155680, n24.000 266119 768453 119932 754228 98606 632686 234274 87299 386093 474154 1284577 155680, 754228, 754228, 754228, 754228, 754228, 754228], 59972, 679677, 519769, 437265 179731 58653 185433, [769062 414047 1078259, 769062 414047 1078259, 769062 414047 1078259, 769062 414047 1078259, 769062 414047 1078259, 769062 414047 1078259, 769062 414047 1078259, 769062 414047 1078259, 769062 414047 1078259, 769062 414047 1078259, 769062 414047 1078259 769062, 769062 414047 1078259 769062, 769062 414047 1078259 769062, 769062 414047 1078259 769062, 769062 414047 1078259 769062, 769062 414047 1078259 769062], [1066831 n21600.000 1066831 n22800.000 1066831 n10200.000 1066831 n10800.000, 1066831 n21600.000 1066831 n22800.000 1066831 n10200.000 1066831 n10800.000, 1066831 n21600.000 1066831 n22800.000 1066831 n10200.000 1066831 n10800.000, 1066831 n21600.000 1066831 n22800.000 1066831 n10200.000 1066831 n10800.000, 1066831 n21600.000 1066831 n22800.000 1066831 n10200.000 1066831 n10800.000, 1066831 n21600.000 1066831 n22800.000 1066831 n10200.000 1066831 n10800.000, 1066831 n21600.000 1066831 n22800.000 1066831 n10200.000 1066831 n10800.000, 1066831 n21600.000 1066831 n22800.000 1066831 n10200.000 1066831 n10800.000, 1066831 n21600.000 1066831 n22800.000 1066831 n10200.000 1066831 n10800.000, 1066831 n21600.000 1066831 n22800.000 1066831 n10200.000 1066831 n10800.000, 1066831 n10200.000, 1066831 n10200.000, 1066831 n10200.000, 1066831 n10200.000, 1066831 n10200.000, 1066831 n10200.000], 780351, 780351 820477 437265 179731 58653 185433 611917 1242195 437265 179731 251307 863335 347067 988697 266119 519519 30603 386817 990716 312815 1242195 31552 1113810 n9984.000 421638 537209 149279 599418 174342 30603 619143 1128577 876603 47299, 125375, [39392, 39392, 39392, 39392, 39392, 39392, 39392, 39392, 39392, 39392, 973211, 973211, 973211, 973211, 973211, 973211], 780351, 1314495 237874 536050 1323749, 1297729 n36.000 n60.000 1178208 n48.000 606827, 820477, 1078259 50164, 150169 435459 16718, 769062, 780351, n1464.000 495356, 1285872, 365413, n61800.000, 820477 437265 179731 58653 185433 n48.000 1187104, 820477, [437265 179731 58653 185433 n48.000 1187104 231473 59972, 437265 179731 58653 185433 n48.000 1187104 231473 59972, 437265 179731 58653 185433 n48.000 1187104 231473, 437265 179731 58653 185433 n48.000 1187104 231473, 437265 179731 58653 185433 n48.000 1187104 231473, 437265 179731 58653 185433 n48.000

In [33]:
product_data['merged_properties'] = [
    {key: value for key, value in zip(row['properties'], row['property_values'])}
    for _, row in product_data.iterrows()
]

# Output the resulting dictionary
display(product_data)


itemid categoryid available  \
0     309858        799         0   
1      89393        491         0   
2      88277       1576         0   
3      90911        179         0   
4      72133       1694         0   
...      ...        ...       ...   
9995  171847        508         0   
9996  271852        624         0   
9997  206063        352         0   
9998   99270       1355         0   
9999  382713       1512         0   

                                                                                                                                                                                                     properties  \
0                                                                             [1066, 112, 159, 202, 227, 283, 364, 384, 400, 566, 595, 6, 678, 689, 698, 764, 776, 790, 839, 853, 888, 917, 934, 964, 992, 997]   
1                                                           [0, 1036, 112, 159, 202, 208, 225, 227, 283, 364, 451, 6, 653, 663, 678, 681, 689, 696, 698, 761, 764, 776, 790, 794, 839, 888, 893, 917, 936, 969]   
2                                                                                      [1010, 1036, 1064, 112, 159, 202, 227, 252, 283, 33, 364, 6, 678, 689, 693, 696, 698, 764, 776, 790, 839, 888, 917, 981]   
3                                                         [112, 159, 19, 202, 213, 227, 243, 28, 283, 364, 38, 434, 441, 46, 480, 575, 6, 678, 689, 698, 713, 764, 776, 790, 810, 839, 888, 917, 960, 961, 994]   
4     [1032, 1037, 1079, 1090, 112, 120, 159, 186, 189, 202, 227, 231, 28, 283, 355, 364, 408, 422, 464, 496, 580, 581, 6, 678, 698, 701, 764, 765, 776, 783, 790, 809, 815, 839, 888, 892, 917, 978, 981, 988]   
...                                                                                                                                                                                                         ...   
9995                                                                          [112, 159, 19, 202, 227, 243, 28, 283, 364, 38, 434, 441, 46, 575, 6, 678, 689, 698, 713, 764, 776, 790, 810, 839, 888, 917, 960]   
9996            [112, 119, 122, 141, 142, 159, 176, 202, 227, 278, 283, 297, 322, 348, 364, 406, 545, 585, 6, 648, 66, 678, 707, 747, 764, 776, 790, 807, 83, 839, 851, 854, 888, 898, 917, 928, 939, 974, 982]   
9997                                                                                                                  [1036, 112, 159, 202, 227, 283, 364, 6, 678, 689, 698, 713, 764, 776, 790, 839, 888, 917]   
9998                  [112, 159, 202, 227, 28, 283, 293, 30, 348, 364, 405, 415, 546, 553, 584, 6, 628, 637, 646, 658, 675, 678, 74, 753, 764, 776, 790, 837, 839, 846, 847, 867, 888, 917, 928, 935, 941, 964]   
9999                                                                                                    [112, 159, 202, 283, 348, 364, 6, 660, 678, 689, 713, 764, 776, 790, 797, 810, 839, 888, 917, 928, 961]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [34]:
def preprocess_text(text):
    # Convert to lowercase and remove special characters
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

def combine_columns(row):
    return preprocess_text (
        "ID: " + str(row.itemid) + "; " +
        "categoryid: " + str(row.categoryid) + "; " +
        "available: " + row.available + "; " +
        "properties: " + str(row.merged_properties) + "; " 
    )


product_data_embeddings = product_data[['itemid']].copy()
product_data_embeddings["combined"] = product_data.apply(lambda row: combine_columns(row), axis=1)
display(product_data_embeddings)

itemid  \
0     309858   
1      89393   
2      88277   
3      90911   
4      72133   
...      ...   
9995  171847   
9996  271852   
9997  206063   
9998   99270   
9999  382713   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [35]:
vectorizer = TextVectorization(output_mode='int', output_sequence_length=10)

# Apply the custom preprocessing function to the 'comb
# Update the vocabulary based on the preprocessed text
vectorizer.adapt(product_data_embeddings['combined'].values)

In [36]:
import os
import requests
import zipfile
import numpy as np

def download_and_extract_glove(glove_url, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    glove_zip = os.path.join(save_dir, 'glove.6B.zip')
    glove_txt = os.path.join(save_dir, 'glove.6B.100d.txt')

    if not os.path.exists(glove_txt):
        print("Downloading GloVe embeddings...")
        response = requests.get(glove_url, stream=True)
        with open(glove_zip, 'wb') as f:
            f.write(response.content)

        print("Extracting GloVe embeddings...")
        with zipfile.ZipFile(glove_zip, 'r') as zip_ref:
            zip_ref.extractall(save_dir)

        os.remove(glove_zip)

# Replace 'save_directory' with the path to the directory where you want to save the embeddings
save_directory = '../gabor'
glove_url = 'https://nlp.stanford.edu/data/glove.6B.zip'
download_and_extract_glove(glove_url, save_directory)

In [37]:
glove_txt = os.path.join(save_directory, 'glove.6B.100d.txt')
embeddings_index = {}
with open(glove_txt, encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Define the Embedding layer using the pre-trained GloVe embeddings
vocab_size = len(vectorizer.get_vocabulary())
embedding_dim = 100  # You can choose the desired embedding dimension
for token in vectorizer.get_vocabulary():
    if token.isnumeric():  # Check if the token is a number
        # Handle out-of-vocabulary numeric tokens
        # This example assigns a random vector to each numeric token
        embeddings_index[token] = np.random.rand(embedding_dim)

embedding_matrix = np.zeros((vocab_size, embedding_dim))
word_index = {word: index for index, word in enumerate(vectorizer.get_vocabulary())}

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    else:
        # If a specific number isn't in GloVe, assign a random vector (or handle as needed)
        embedding_matrix[i] = np.random.rand(embedding_dim)

embedding_layer = Embedding(
    vocab_size,
    embedding_dim,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    trainable=False
)



In [38]:
# Convert the preprocessed text to numeric vectors
numeric_vectors = vectorizer(product_data_embeddings['combined'].values)

# Get word embeddings
word_embeddings = embedding_layer(numeric_vectors)
word_embeddings, word_embeddings.shape

(<tf.Tensor: shape=(10000, 10, 100), dtype=float32, numpy=
 array([[[-0.22458   , -0.58967   ,  0.43007   , ..., -0.63613   ,
           0.31284   , -0.82294   ],
         [ 0.34483272,  0.89599746,  0.32817388, ...,  0.07255627,
           0.8304072 ,  0.4393786 ],
         [ 0.43086272,  0.6279726 ,  0.6842533 , ...,  0.928418  ,
           0.5640103 ,  0.12308144],
         ...,
         [ 0.0241208 ,  0.35660684,  0.94197994, ...,  0.86376643,
           0.303241  ,  0.2674872 ],
         [ 0.5561478 ,  0.40737036,  0.7586439 , ...,  0.28461322,
           0.36693388,  0.29774147],
         [ 0.00935586,  0.8938262 ,  0.5759318 , ...,  0.5674046 ,
           0.05063055,  0.71110153]],
 
        [[-0.22458   , -0.58967   ,  0.43007   , ..., -0.63613   ,
           0.31284   , -0.82294   ],
         [ 0.8959463 ,  0.28381962,  0.35820735, ...,  0.8701696 ,
           0.3265165 ,  0.4407453 ],
         [ 0.43086272,  0.6279726 ,  0.6842533 , ...,  0.928418  ,
           0.5640103 ,  0

In [39]:
# def cosine_similarity(a, b):
#     return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
from sklearn.metrics.pairwise import cosine_similarity

def get_n_nearest_neighbors(query_embedding, neighbors_embedding, n = 5, ):
    input_embedding_flat = query_embedding.numpy().reshape(query_embedding.shape[0], -1)
    word_embeddings_flat = neighbors_embedding.numpy().reshape(neighbors_embedding.shape[0], -1)
    # Calculate the cosine similarity between the input embedding and all other word embeddings
    similarities = cosine_similarity(input_embedding_flat, word_embeddings_flat)
    # Get the indices of the most similar neighbors
    neighbor_indices = np.argsort(similarities[0])[::-1][:n]

    scores = [similarities[0][i] for i in neighbor_indices]

    return neighbor_indices, scores


indices = get_n_nearest_neighbors(tf.gather(word_embeddings, [0]), word_embeddings)
display(indices)

(array([   0, 2597, 1199, 2675, 6727]),
 [1.0, 0.9512992, 0.9482227, 0.94338334, 0.94296706])

In [40]:
product_data_embeddings.iloc[indices[0]].combined

0                                                                                                                                                                                                                                                                                           id 309858 categoryid 799 available 0 properties 1066 n1260000 424566 112 679677 159 519769 202 125782 n1440000 822075 227 1308244 283 1308244 590597 125782 n1440000 822075 364 909382 384 726612 400 n586800 424566 566 n619200 424566 595 873905 6 1308244 678 590597 689 150169 274770 79212 698 1308244 764 1285872 776 1081162 790 n161400000 839 590597 853 826838 888 125782 n1440000 822075 917 1301157 n1440000 822075 934 638115 964 n2640000 1128577 992 n21600 404373 997 n360000 863188 
2597                                                                                                                                                                                                                                  

In [41]:
product_data.iloc[indices[0]]

,itemid,categoryid,available,properties,property_values,merged_properties
0,309858,799,0,"[1066, 112, 159, 202, 227, 283, 364, 384, 400, 566, 595, 6, 678, 689, 698, 764, 776, 790, 839, 853, 888, 917, 934, 964, 992, 997]","[n1260.000 424566, 679677, 519769, 125782 n1440.000 822075, 1308244, 1308244 590597 125782 n1440.000 822075, 909382, 726612, n586.800 424566, n619.200 424566, 873905, 1308244, 590597, 150169 274770 79212, 1308244, 1285872, 1081162, n161400.000, 590597, 826838, 125782 n1440.000 822075, 1301157 n1440.000 822075, 638115, n2640.000 1128577, n21.600 404373, n360.000 863188]","{'1066': 'n1260.000 424566', '112': '679677', '159': '519769', '202': '125782 n1440.000 822075', '227': '1308244', '283': '1308244 590597 125782 n1440.000 822075', '364': '909382', '384': '726612', '400': 'n586.800 424566', '566': 'n619.200 424566', '595': '873905', '6': '1308244', '678': '590597', '689': '150169 274770 79212', '698': '1308244', '764': '1285872', '776': '1081162', '790': 'n161400.000', '839': '590597', '853': '826838', '888': '125782 n1440.000 822075', '917': '1301157 n1440.000 822075', '934': '638115', '964': 'n2640.000 1128577', '992': 'n21.600 404373', '997': 'n360.000 863188'}"
2597,128085,799,0,"[1066, 112, 124, 134, 159, 202, 227, 283, 364, 400, 465, 566, 595, 6, 672, 678, 689, 698, 764, 776, 790, 839, 853, 888, 909, 917, 934, 964, 992]","[n1020.000 424566, 679677, 535199, n12.000, 519769, 1285833 982806, 1308244, 1308244 1115724 1285833 982806, 309729, n1060.800 424566, n288.000 964443, n884.400 424566, 1010870, 1308244, 805298, 1115724, 150169 274770 79212, 1308244, 1285872, 1238499, n659160.000, 1115724, 1276304, 1285833 982806, n5640.000, n92585160672.000, 362953 683, n2760.000 1128577, n612.000 404373]","{'1066': 'n1020.000 424566', '112': '679677', '124': '535199', '134': 'n12.000', '159': '519769', '202': '1285833 982806', '227': '1308244', '283': '1308244 1115724 1285833 982806', '364': '309729', '400': 'n1060.800 424566', '465': 'n288.000 964443', '566': 'n884.400 424566', '595': '1010870', '6': '1308244', '672': '805298', '678': '1115724', '689': '150169 274770 79212', '698': '1308244', '764': '1285872', '776': '1238499', '790': 'n659160.000', '839': '1115724', '853': '1276304', '888': '1285833 982806', '909': 'n5640.000', '917': 'n92585160672.000', '934': '362953 683', '964': 'n2760.000 1128577', '992': 'n612.000 404373'}"
1199,158206,799,0,"[1066, 112, 124, 134, 159, 202, 227, 283, 364, 400, 465, 566, 595, 6, 678, 689, 698, 764, 776, 790, 839, 853, 888, 90, 917, 934, 964, 992, 997]","[n474.000 424566, 679677, 535199, n12.000, 519769, 592704, 1308244, 1308244 242942 592704, 38759, n324.000 424566, n7.200 547016, n78.000 424566, 873905, 1308244, 242942, 150169 274770 79212, 1308244, 1285872, 1024210, n42240.000, 242942, 1276304, 592704, 417679, 592704, 638115, n2640.000 1128577, n72.000 404373, n54.000 1333885]","{'1066': 'n474.000 424566', '112': '679677', '124': '535199', '134': 'n12.000', '159': '519769', '202': '592704', '227': '1308244', '283': '1308244 242942 592704', '364': '38759', '400': 'n324.000 424566', '465': 'n7.200 547016', '566': 'n78.000 424566', '595': '873905', '6': '1308244', '678': '242942', '689': '150169 274770 79212', '698': '1308244', '764': '1285872', '776': '1024210', '790': 'n42240.000', '839': '242942', '853': '1276304', '888': '592704', '90': '417679', '917': '592704', '934': '638115', '964': 'n2640.000 1128577', '992': 'n72.000 404373', '997': 'n54.000 1333885'}"
2675,311974,799,0,"[1066, 112, 124, 159, 202, 227, 283, 364, 400, 566, 595, 6, 678, 689, 698, 764, 776, 790, 839, 853, 888, 90, 917, 964, 992, 997]","[n171.600 424566, 679677, 539823, 519769, 1293886 n6840.000, 1308244, 1308244 663739 1293886 n6840.000, 609600, n228.000 424566, n98.400 424566, 873905, 1308244, 663739, 150169 274770 79212, 1308244, 1285872, [883137, 883137, 883137, 883137, 883137, 883137, 883137, 883137, 883137, 883137, 883137, 883137, 883137, 883137, 272614, 272614, 272614, 272614], n69120.000, 663739, 1

In [42]:
def evaluate_coverage_tensor(embeddings_tensor, n=5):
    recommended_items = set()
    scores_list = []
    total_items = embeddings_tensor.shape[0]

    for i in range(total_items):
        # Exclude the current query embedding from the neighbors
        neighbors_embeddings = tf.concat([embeddings_tensor[:i], embeddings_tensor[i+1:]], axis=0)

        query_embedding = embeddings_tensor[i:i+1]
        indices, scores = get_n_nearest_neighbors(query_embedding, neighbors_embeddings, n)

        # Convert indices and scores to numpy arrays if they are TensorFlow tensors
        if isinstance(indices, tf.Tensor):
            indices = indices.numpy().flatten()
        if isinstance(scores, tf.Tensor):
            scores = scores.numpy()

        recommended_items.update(indices)
        scores_list.extend(scores)
        if i % 1000 == 0:
            print(i)

    coverage_score = len(recommended_items) / (total_items - 1)

    return coverage_score, scores_list

# Example usage
coverage_score, confidence_scores = evaluate_coverage_tensor(word_embeddings)
print(f"Coverage Score: {coverage_score}")
print(f"Confidence Score: {np.mean(confidence_scores)}; Min Score: {np.min(confidence_scores)}; Max Score: {np.max(confidence_scores)}")


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
Coverage Score: 0.9665966596659666
Confidence Score: 0.9713382124900818; Min Score: 0.8849564790725708; Max Score: 0.9864251017570496
